# Birliktelik Kuralı Madenciliği - Mlxtend ile Pazar Sepeti Analizi

# 1. Giriş

<img width="1000" height="400" alt="netflix" align="left" src="https://user-images.githubusercontent.com/36535914/78720348-369df700-792e-11ea-9950-8f5731409171.png">
***

Birliktelik kuralı öğrenme, büyük veritabanlarındaki değişkenler arasındaki ilişkileri keşfetmek için kural tabanlı bir makine öğrenme yöntemidir. Amaç, güven (confidence) veya lift gibi bazı ölçüleri kullanarak veri kümelerinde keşfedilen güçlü ilişkileri belirlemektir.

Tüketici davranışına dayalı daha somut bir örnekte olduğu gibi, çocuk bezi satın alan kişilerin aynı zamanda bira da satın alma olasılığının yüksek olduğunu öne süren {Çocuk Bezi}→{Bira} gibi ilişkilerin yakalanmasına sebep olur. Böyle bir birliktelik kuralının "ilgisini" değerlendirmek için farklı metrikler geliştirilmiştir. Mevcut uygulama, yukarıda bahsettiğimiz güven ve lift metriklerini kullanıyor


- ***Bir müşteri ekmek satın alırsa, süt satın alma olasılığı %70'tir.***

Yukarıdaki birliktelik kuralında ekmek girdi, süt ise sonuçtur. Basitçe söylemek gerekirse, müşterilerini daha iyi hedeflemek için bir perakende mağazasının birliktelik kuralı olarak anlaşılabilir. Yukarıdaki kural, bazı veri kümelerinin kapsamlı bir analizinin bir sonucuysa, yalnızca müşteri hizmetlerini iyileştirmek için değil, aynı zamanda şirketin gelirini de iyileştirmek için kullanılabilir.

Yukarıdakilere ek olarak, tavsiye sistemleri, tıbbi teşhis, protein dizisi, nüfus sayımı verileri ve hatta suç önleme gibi birçok uygulamada birliktelik kuralına dayalı öğrenme teknikleri de kullanılmaktadır.

# 2. Veri Yükleme

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules
import matplotlib.pyplot as plt
plt.style.use('default')

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day7-DecisionTree/notebooks"
ROOT_DIR = "https://raw.githubusercontent.com/yapay-ogrenme/casgem-eu-project-training-on-data-mining/main/PART1/Day8-UnsupervisedLearning/notebooks/"

DATASET_PATH = ROOT_DIR + "/datasets/"

In [ ]:
data = pd.read_csv(DATASET_PATH + "Market_Basket_Optimisation.csv", header=None)
data.shape

In [ ]:
data

In [ ]:
data.describe()

# 3. Veri Görselleştirme

***- Veri kümesinde en çok talep edilen öğeler / Top10***

In [ ]:
# 1. Gather All Items of Each Transactions into Numpy Array
transaction = []
for i in range(0, data.shape[0]):
    for j in range(0, data.shape[1]):
        transaction.append(data.values[i,j])

transaction = np.array(transaction)

# 2. Transform Them a Pandas DataFrame
df = pd.DataFrame(transaction, columns=["items"]) 
df["incident_count"] = 1 # Put 1 to Each Item For Making Countable Table, to be able to perform Group By

# 3. Delete NaN Items from Dataset
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)

# 4. Final Step: Make a New Appropriate Pandas DataFrame for Visualizations  
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()

# 5. Initial Visualizations
df_table.head(10).style.background_gradient(cmap='Blues')

***- Veri kümesinde en çok talep edilen öğeler / Top30***

In [ ]:
df_table["all"] = "all" # to have a same origin

fig = px.treemap(df_table.head(30), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(30), hover_data=['items'],
                  color_continuous_scale='Blues',
                  )
fig.show()

***- Bir işlemde ürünlerin birden fazla kaydı olup olmadığını kontrol edelim.***<br>
***- Cevap "Evet" ise, sonraki adımlarda apriori algoritmasını yanlış yönlendirebilecekleri için bunları ele almamız gerekir.***

In [ ]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array
# By Doing So, We Will Be Able To Iterate Through Array of Transactions

transaction = []
for i in range(data.shape[0]):
    transaction.append([str(data.values[i,j]) for j in range(data.shape[1])])
    
transaction = np.array(transaction)

# Create a DataFrame In Order To Check Status of Top20 Items

top20 = df_table["items"].head(20).values
array = []
df_top20_multiple_record_check = pd.DataFrame(columns=top20)

for i in range(0, len(top20)):
    array = []
    for j in range(0,transaction.shape[0]):
        array.append(np.count_nonzero(transaction[j]==top20[i]))
        if len(array) == len(data):
            df_top20_multiple_record_check[top20[i]] = array
        else:
            continue
            

df_top20_multiple_record_check.head(10)


In [ ]:
df_top20_multiple_record_check.describe()

- ***Yukarıda gördüğünüz gibi sadece Chocolate'ın max değeri 2'dir. Diğerlerinin max değeri 1'dir. Bu nedenle verilerin homojen olduğunu söyleyebiliriz, herhangi bir çıkarım yapmadan ilerleyebiliriz.***

***- Seçim Analizi / Müşterilerin İlk Tercihleri***

In [ ]:
# 1. Gather Only First Choice of Each Transactions into Numpy Array
# Similar Pattern to Above, Only Change is the Column Number "0" in Append Function
transaction = []
for i in range(0, data.shape[0]):
    transaction.append(data.values[i,0])

transaction = np.array(transaction)

# 2. Transform Them a Pandas DataFrame
df_first = pd.DataFrame(transaction, columns=["items"])
df_first["incident_count"] = 1

# 3. Delete NaN Items from Dataset
indexNames = df_first[df_first['items'] == "nan" ].index
df_first.drop(indexNames , inplace=True)

# 4. Final Step: Make a New Appropriate Pandas DataFrame for Visualizations  
df_table_first = df_first.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_first["food"] = "food"
df_table_first = df_table_first.truncate(before=-1, after=15) # Fist 15 Choice
df_table_first

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (20, 20)

first_choice = nx.from_pandas_edgelist(df_table_first, source = 'food', target = "items", edge_attr = True)
pos = nx.spring_layout(first_choice)

nx.draw_networkx_nodes(first_choice, pos, node_size = 12500, node_color = "lavender")
nx.draw_networkx_edges(first_choice, pos, width = 3, alpha = 0.6, edge_color = 'black')
nx.draw_networkx_labels(first_choice, pos, font_size = 18, font_family = 'sans-serif')
plt.axis('off')
plt.grid()
plt.title('Top 15 First Choices', fontsize = 25)
plt.show()

***- Seçim Analizi / Müşterilerin İkinci Tercihleri***

In [ ]:
# 1. Gather Only Second Choice of Each Transaction into Numpy Array

transaction = []
for i in range(0, data.shape[0]):
    transaction.append(data.values[i,1])

transaction = np.array(transaction)

# 2. Transform Them a Pandas DataFrame
df_second = pd.DataFrame(transaction, columns=["items"]) 
df_second["incident_count"] = 1

# 3. Delete NaN Items from Dataset
indexNames = df_second[df_second['items'] == "nan" ].index
df_second.drop(indexNames , inplace=True)

# 4. Final Step: Make a New Appropriate Pandas DataFrame for Visualizations  
df_table_second = df_second.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_second["food"] = "food"
df_table_second = df_table_second.truncate(before=-1, after=15) # Fist 15 Choice
df_table_second

In [ ]:
import warnings
warnings.filterwarnings('ignore')

second_choice = nx.from_pandas_edgelist(df_table_second, source = 'food', target = "items", edge_attr = True)
pos = nx.spring_layout(second_choice)
nx.draw_networkx_nodes(second_choice, pos, node_size = 12500, node_color = "honeydew")
nx.draw_networkx_edges(second_choice, pos, width = 3, alpha = 0.6, edge_color = 'black')
nx.draw_networkx_labels(second_choice, pos, font_size = 18, font_family = 'sans-serif')
plt.rcParams['figure.figsize'] = (20, 20)
plt.axis('off')
plt.grid()
plt.title('Top 15 Second Choices', fontsize = 25)
plt.show()

***- Seçim Analizi / Müşterilerin Üçüncü Tercihleri***

In [ ]:
# 1. Gather Only Third Choice of Each Transaction into Numpy Array
## For Column "2"
transaction = []
for i in range(0, data.shape[0]):
    transaction.append(data.values[i,2])

transaction = np.array(transaction)

# 2. Transform Them a Pandas DataFrame
df_third = pd.DataFrame(transaction, columns=["items"]) # Transaction Item Name
df_third["incident_count"] = 1 # Put 1 to Each Item For Making Countable Table, Group By Will Be Done Later On

# 3. Delete NaN Items from Dataset
indexNames = df_third[df_third['items'] == "nan" ].index
df_third.drop(indexNames , inplace=True)

# 4. Final Step: Make a New Appropriate Pandas DataFrame for Visualizations  
df_table_third = df_third.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()
df_table_third["food"] = "food"
df_table_third = df_table_third.truncate(before=-1, after=15) # Fist 15 Choice
df_table_third

In [ ]:
fig = go.Figure(data=[go.Bar(x=df_table_third["items"], y=df_table_third["incident_count"],
            hovertext=df_table_third["items"], text=df_table_third["incident_count"], textposition="outside")])

fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.65)
fig.update_layout(title_text="Customers' Third Choices", template="plotly_dark")
fig.show()

# 4. Veri Ön işleme

***Apriori algoritmasını kullanabilmek ve en sık öğe kümelerini elde edebilmek için veri kümemizi satırların işlem, sütunların ise ürün olduğu 1 – 0 matrisine dönüştürmemiz gerekir. Bu matriste, o işlemde ürün alınmışsa “1”, o işlemde ürün satın alınmamışsa “0” kodlanmalıdır. Bu ön işleme algoritmanın kullanımı için gereklidir.***



In [ ]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array

transaction = []
for i in range(data.shape[0]):
    transaction.append([str(data.values[i,j]) for j in range(data.shape[1])])
    
transaction = np.array(transaction)
transaction

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset

In [ ]:
dataset.shape

***Şu anda 121 sütunumuz/özelliğimiz var. 121 özelliğinden en sık görülen öğe kümelerini çıkarmak, başlangıç için zorlayıcı olacaktır*** <br>
***Bu nedenle, Bölüm-3'te zaten gösterilen İlk 50 öğe ile başlayacağız***



In [ ]:
first50 = df_table["items"].head(50).values # Select Top50
dataset = dataset.loc[:,first50] # Extract Top50
dataset

In [ ]:
dataset.columns
# We extracted first 50 column successfully.

In [ ]:
# Convert dataset into 1-0 encoding

def encode_units(x):
    if x == False:
        return 0 
    if x == True:
        return 1
    
dataset = dataset.applymap(encode_units)
dataset.head(10)

# 5. Algoritmanın Uygulaması

## 5.1. Birliktelik Kurallarının Temel Kavramları / Apriori Algoritması

X: 1.ürün Y: 2.ürün N: Toplam Alışveriş

**- Destek (Support) : Ürünlerin Birlikte Geçme Olasılığı**

Support(X, Y) = Freq(X,Y)/N

(X ve Y farklı ürünlerinin birlikte geçme frekansı / Toplam Alışveriş)

Support değerini hesaplama amacımız eşik değeri belirlemek. Veri setine bakıldığında bir çok ürün beraber görülebilir eşik değerinin altında kalan ürünlerde eleme yapacağız.

**- Güven (Confidence) : X’i Alanların Y’yi Alma Olasılığı**

Confidence(X, Y) = Freq(X,Y)/Freq(X)

(X ve Y farklı ürünlerinin birlikte geçme frekansı / X’in gözlenme frekansı)

Analiz yapıldığında ister support değerine göre ister confidence değerine göre eşik değeri belirlenebilir. 

Eşik değerini gerçek hayat verilerinde çok az alabiliriz.

**- Lift = Support(X,Y)/(Support(X)*Support(Y))**

X ürünü alanların Y ürünü satın alması şu kadar kat artıyor yorumu vardır.

## 5.2. Implementation

**Birliktelik Kuralları Öğrenme uygulamaları için en yaygın kullanılan kitaplık 'Mlxtend'dir. Biz de o kütüphaneyi kullanacağız.**

In [ ]:
# Extracting the most frequest itemsets via Mlxtend.
# The length column has been added to increase ease of filtering.

frequent_itemsets = apriori(dataset, min_support=0.01, use_colnames=True)

frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

***Öğe kümelerini aşağıdaki gibi kolayca keşfedebiliriz***

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.05) ]

In [ ]:
frequent_itemsets[ (frequent_itemsets['length'] == 3) ].head()

***Şimdi, kural oluşturmada çıkarılan sık öğe kümelerini kullanacağız.***

In [ ]:
# We can create our rules by defining metric and its threshold.

# For a start, 
#      We set our metric as "Lift" to define whether antecedents & consequents are dependent our not.
#      Treshold is selected as "1.2" since it is required to have lift scores above than 1 if there is dependency.

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)

rules["antecedents_length"] = rules["antecedents"].apply(lambda x: len(x))
rules["consequents_length"] = rules["consequents"].apply(lambda x: len(x))

rules.sort_values("lift",ascending=False)

***Yukarıdaki tabloya göre, lift puanı eşiğin yaklaşık 2.5 katı olduğu ve güven puanının umut verici (%32) olması nedeniyle (ot-biber) ve (dana kıyma) arasındaki bağımlılığın yüksek olduğunu rahatlıkla söyleyebiliriz.***


***Verilerden daha fazla içgörü elde etmek için güvene bakalım!***

In [ ]:
# Sort values based on confidence

rules.sort_values("confidence",ascending=False)

- ***Yukarıdaki tabloya göre (yumurta, kıyma) alan müşterilerin %50 olasılıkla (güven) ile (maden suyu) alması beklenmektedir. Lift puanı da bu hipotezi destekliyor***
- ***Satışları artırmak için onları yakın tutmak daha iyi olur!***

***Veri setinde en çok talep edilen ürün maden suyu olduğu için ilişkilendirme sonuçlarına ağırlıklı olarak maden suyu hakimdir. Bu nedenle, daha fazla içgörü elde etmek için maden suyu hariç bir güven tablosu oluşturmak daha iyidir***

In [ ]:
rules[~rules["consequents"].str.contains("mineral water", regex=False) & 
      ~rules["antecedents"].str.contains("mineral water", regex=False)].sort_values("confidence", ascending=False).head(10)

***Yukarıdaki maden suyu hariç tutulan tabloya göre kıyma ile spagetti, kırmızı şarap ve spagetti arasında önemli bir ilişki olduğunu söyleyebiliriz. Lift puanları da bunu destekliyor***

***Fark etmiş olabileceğiniz gibi hem maden suyu dahil hem de hariç tablosunda kıyma en üstte. Bu nedenle, kıyma ile ilgili yeni ilişkiler yakalamak ve satışları artırmak için kıymanın öncül olduğu ilişkilere bakalım.***

In [ ]:
rules[rules["antecedents"].str.contains("ground beef", regex=False) & rules["antecedents_length"] == 1].sort_values("confidence", ascending=False).head(10)

- ***Güven ve lift puanı yüksek birçok ilişki
 vardır. Doğru yoldayız!***

# 6. Sonuçlar

***Yukarıdaki incelemelerde gördüğünüz gibi, algoritmanın ve mlxtend kitaplığının esnekliği yüksektir, bu nedenle farklı yönleri kolayca araştırabilir ve verilerden yeni ilişkilendirmeler elde edebiliriz. Bu nedenle, diğer ürünler (Top50'nin geri kalanı) hesaplamaya dahil edilerek veya kriter eşiği değiştirilerek araştırmalar daha ayrıntılı hale getirilebilir. Bununla birlikte, birliktelik kuralı öğrenme yinelemeli bir şemaya sahip olduğundan, verileri anlama ve yorumlama becerileri ve etkinlikleri gerçekten önemlidir. Bu durumda, doğru yolda olduğumuzdan emin olmak için veri görselleştirme ve/veya veri temizleme (gerekirse) adımlarına yeterince önem vermeliyiz.***